# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities = pd.read_csv("output_data/cities.csv")
cities.head()

,Name,ID,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,majene,1636806.0,-3.54,118.97,75.61,91.0,95.0,3.76
1,busselton,2075265.0,-33.65,115.33,71.01,38.0,0.0,9.40
2,turukhansk,1488903.0,65.82,87.98,-1.41,96.0,100.0,6.55
3,ulladulla,2145554.0,-35.35,150.47,69.80,83.0,90.0,6.93
4,manzhouli,2035836.0,49.60,117.43,-5.04,100.0,73.0,2.42


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [17]:
gmaps.configure(api_key=g_key)
locations = cities[["Lat", "Lng"]]
humidity = cities["Humidity"]

In [19]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=75,
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#I chose ideal the ideal city weather based on what I like, coldish and cloudy.

narrowed_city_df = cities.loc[((cities["Max Temp"]<50) & (cities["Max Temp"]>30)) & (cities["Wind Speed"]<=8) & \
                                (cities["Cloudiness"]>60)]
narrowed_city_df = narrowed_city_df.dropna()
narrowed_city_df

,Name,ID,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
58,nikolskoye,546105.0,59.70,30.79,33.80,80.0,75.0,6.71
72,goderich,5962442.0,43.75,-81.72,36.00,83.0,100.0,4.00
73,hasaki,2112802.0,35.73,140.83,36.00,62.0,75.0,5.82
79,namie,2111704.0,37.48,141.00,34.00,82.0,93.0,7.76
132,twin falls,5610810.0,42.56,-114.46,42.80,93.0,90.0,5.82
133,fangshan,1811542.0,39.70,115.99,34.00,73.0,100.0,4.47
151,pemberton,6100799.0,50.32,-122.80,33.80,100.0,90.0,3.11
156,parchim,2855441.0,53.43,11.85,34.00,85.0,75.0,6.93
180,shiyan,1794903.0,32.65,110.78,30.47,96.0,61.0,3.04
193,luan,2659977.0,46.36,6.98,39.20,74.0,75.0,3.02


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = pd.DataFrame(columns = ["Hotel Name"])

hotel_df["Hotel Name"]= ""
hotel_df["Lat"] = ""
hotel_df["Lng"] = ""
hotel_df["City"] = ""
hotel_df["Country"]= ""
hotel_df["PlaceID"]=""
hotel_df.head()

,Hotel Name,Lat,Lng,City,Country,PlaceID


In [7]:
#test for single call

target_coordinates = "41.37, -73.41"
target_search = "lodging"
target_radius = 5000


# set up a parameters dictionary
params = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "key": g_key
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
response = requests.get(base_url, params=params)

# convert response to json
cityHotel = response.json()
print(json.dumps(cityHotel, indent=4, sort_keys=True))

{
    "html_attributions": [],
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 41.4096498,
                    "lng": -73.4112932
                },
                "viewport": {
                    "northeast": {
                        "lat": 41.41104127989272,
                        "lng": -73.41005947010729
                    },
                    "southwest": {
                        "lat": 41.40834162010727,
                        "lng": -73.41275912989273
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "id": "f6ae9c26211d58e5c120f75e16e54c827c83044c",
            "name": "Hampton Inn Danbury",
            "opening_hours": {
                "open_now": true
            },
            "photos": [
                {
                    "height": 1500,
                    "html_attributions": [
                  

In [7]:
for index, row in narrowed_city_df.iterrows():
    
    lat = row["Lat"]
    lng = row["Lng"]
    
    # geocoordinates
    target_coordinates = f"{lat}, {lng}"
    target_search = "lodging"
    target_radius = 5000


    # set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "keyword": target_search,
        "radius": target_radius,
        "key": g_key
    }

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params)

    # convert response to json
    cityHotel = response.json()
    try:
        hotel_df.loc[index, "Hotel Name"] = cityHotel["results"][0]["name"]
        hotel_df.loc[index, "Lat"] = cityHotel["results"][0]["geometry"]["location"]["lat"]
        hotel_df.loc[index, "Lng"] = cityHotel["results"][0]["geometry"]["location"]["lng"]
        hotel_df.loc[index, "PlaceID"] = cityHotel["results"][0]["place_id"]
    except:
        print("Ruh-Roh, no data. Skipping...")

Ruh-Roh, no data. Skipping...
Ruh-Roh, no data. Skipping...
Ruh-Roh, no data. Skipping...
Ruh-Roh, no data. Skipping...


In [8]:
hotel_df.head()

,Hotel Name,Lat,Lng,City,Country,PlaceID
58,Tourist House - Sablino,59.6603,30.7785,NaN,NaN,ChIJiWVb2YaOvUYR7NUQLV2IGo0
72,Samuels Hotel,43.7521,-81.6978,NaN,NaN,ChIJiUZATCnDKIgRAuh2Np_JVPA
73,ホテル＆スパ月美 太陽の里,35.7028,140.862,NaN,NaN,ChIJh2dlotw8I2ARQoZiQBMrN04
79,福島いこいの村なみえ,37.48,140.998,NaN,NaN,ChIJNbgPJe7pIGARQEiULUXT684
132,The Fillmore Inn,42.5633,-114.465,NaN,NaN,ChIJv9--ubGjrFQRmv77oXVM9Bk


In [9]:
#finding city name and country
for index, row in hotel_df.iterrows():
    placeid = row["PlaceID"]
    
    params = {
        "place_id":placeid,
        "key": g_key
    }
    
    base = "https://maps.googleapis.com/maps/api/place/details/json"
    
    responseCity = requests.get(base, params=params)
    
    places = responseCity.json()
    
    try:
        hotel_df.loc[index, "City"] = places["result"]["address_components"][3]["long_name"]
        hotel_df.loc[index, "Country"] = places["result"]["address_components"][6]["long_name"]
    except:
        print("Roh-Roh, no data. Skipping...")
        
#try printing the url to see what the layout of one actually looks like instead of just using the google site as a reference.


Roh-Roh, no data. Skipping...
Roh-Roh, no data. Skipping...
Roh-Roh, no data. Skipping...
Roh-Roh, no data. Skipping...
Roh-Roh, no data. Skipping...


In [10]:
hotel_df

,Hotel Name,Lat,Lng,City,Country,PlaceID
58,Tourist House - Sablino,59.6603,30.7785,Tosnensky District,187010,ChIJiWVb2YaOvUYR7NUQLV2IGo0
72,Samuels Hotel,43.7521,-81.6978,Ashfield-Colborne-Wawanosh,Canada,ChIJiUZATCnDKIgRAuh2Np_JVPA
73,ホテル＆スパ月美 太陽の里,35.7028,140.862,Chiba,NaN,ChIJh2dlotw8I2ARQoZiQBMrN04
79,福島いこいの村なみえ,37.48,140.998,大字,福島県,ChIJNbgPJe7pIGARQEiULUXT684
132,The Fillmore Inn,42.5633,-114.465,Twin Falls County,83301,ChIJv9--ubGjrFQRmv77oXVM9Bk
133,Hampton by Hilton,39.7174,116.05,100031,NaN,ChIJxX_CqY898DURnuUgIcSzg9o
151,Pemberton Valley Lodge,50.3149,-122.793,Squamish-Lillooet,V0N 2L1,ChIJuWXzEa81h1QRO4fngt3IH8E
156,Hotel Stadtkrug,53.4265,11.8478,Ludwigslust-Parchim,19370,ChIJC1o6lrScrkcRzbhJBOsl0JE
180,DoubleTree by Hilton Hotel Shiyan,32.6534,110.799,Shiyan Shi,442011,ChIJR2E_K7BehzYRFETMS8RSAU8
193,Chambres d'hôtes et roulottes - Les Noisetiers,46.3521,7.03323,Aigle,1863,ChIJ3VLcPJ6WjkcRyvMxsqMmE9o


In [21]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()] #starter code has narrowed_city_df, but there's not hotel name column in that df.
locations = hotel_df[["Lat", "Lng"]]

In [22]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)


# Display Map

fig = gmaps.figure()
fig.add_layer(marker_layer)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))